# Homework 2 - Recurrent Neural Networks

In this part of the homework we are going to work with Recurrent Neural Networks, in particular GRU. One of the greatest things that Recurrent Neural Networks can do when working with sequences is retaining data from several timesteps in the past. We are going to explore that property by constructing an 'echo' Recurrent Neural Network.

The goal here is to make a model that given a sequence of letters or digits will output that same sequence, but with a certain delay. Let's say the input is a string 'abacaba', we want the model to not output anything for 3 steps (delay length), and then output the original string step by step, except the last 3 characters. So, target output is then 'XXXabac', where 'X' is empty output.

This is similar to [this notebook](https://github.com/Atcold/pytorch-Deep-Learning/blob/master/09-echo_data.ipynb) (which you should refer to when doing this assignment), except we're working not with a binary string, but with a sequence of integers between 0 and some N. In our case N is 26, which is the number of letters in the alphabet.

## Dataset

Let's implement the dataset. In our case, the data is basically infinite, as we can always generate more examples on the fly, so don't need to load anything from disk.

In [ ]:
import random
import string

import torch
import torch.nn as nn

# Max value of the generated integer. 26 is chosen becuase it's
# the number of letters in English alphabet.
N = 26


def idx_to_onehot(x, k=N+1):
  """ Converts the generated integers to one-hot vectors """
  ones = torch.sparse.torch.eye(k)
  shape = x.shape
  res = ones.index_select(0, x.view(-1).type(torch.int64))
  return res.view(*shape, res.shape[-1])


class EchoDataset(torch.utils.data.IterableDataset):

  def __init__(self, delay=4, seq_length=15, size=1000):
    self.delay = delay
    self.seq_length = seq_length
    self.size = size
  
  def __len__(self):
    return self.size

  def __iter__(self):
    """ Iterable dataset doesn't have to implement __getitem__.
        Instead, we only need to implement __iter__ to return
        an iterator (or generator).
    """
    for _ in range(self.size):
      seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
      result = torch.cat((torch.zeros(self.delay), seq[:self.seq_length - self.delay])).type(torch.int64)
      yield seq, result

DELAY = 4
DATASET_SIZE = 200000
ds = EchoDataset(delay=DELAY, size=DATASET_SIZE)

In [ ]:
for a in ds:
  print(a[0])
  print(a[1])
  break

tensor([16, 19, 25, 15, 25,  5,  2, 20, 11, 10, 18, 14, 14, 16,  3])
tensor([ 0,  0,  0,  0, 16, 19, 25, 15, 25,  5,  2, 20, 11, 10, 18])


## Model

Now, we want to implement the model. For our purposes, we want to use GRU. The architecture consists of GRU and a decoder. Decoder is responsible for decoding the GRU hidden state to yield a predicting for the next output. The parts you are responsible for filling with your code are marked with `TODO`. 

In [ ]:
class GRUMemory(torch.nn.Module):

  def __init__(self, hidden_size):
    super().__init__()
    #TODO: initialize your submodules
    self.hidden_size = hidden_size
    self.gru = nn.GRU(
      input_size=N+1,
      hidden_size=hidden_size,
      num_layers=1,
      batch_first=True
    )
    self.fc = nn.Linear(
      in_features=hidden_size,
      out_features=N+1
    )

  def forward(self, x):
    # inputs: x - input tensor of shape (batch_size, seq_length, N+1)
    # returns:
    # logits (scores for softmax) of shape (batch size, seq_length, N + 1)
    # TODO implement forward pass
    h0 = None
    
    output, hn = self.gru(x, h0)  # output:[batch_size, seq_len, hidden_size]
    output = self.fc(output) # [batch_size, seq_len, N+1]
    logits = torch.sigmoid(output)
    return logits # [batch_size, seq_len, N+1]

  @torch.no_grad()
  def test_run(self, s):
    # This function accepts one string s containing lowercase characters a-z. 
    # You need to map those characters to one-hot encodings, 
    # then get the result from your network, and then convert the output 
    # back to a string of the same length, with 0 mapped to ' ', 
    # and 1-26 mapped to a-z.

    # TODO
    x = torch.empty(0)
    for i in range(len(s)):
      tmp = ord(s[i])-96
      x = torch.cat((x,torch.tensor([tmp])))
    input = idx_to_onehot(x, N+1)
    input = input.unsqueeze(0).to(torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))

    logits = self.forward(input) # [1, seq_len, N+1]
    logits = logits.squeeze()
    score, predicted = torch.max(logits, 1)

    res = ""
    for i in range(len(predicted)):
        if predicted[i] == 0:
            res += " "
        else:
            res += chr(predicted[i]+96)
    return res

## Training
Below you need to implement the training of the model. We give you more freedom as for the implementation. The two limitations are that it has to execute within 10 minutes, and that error rate should be below 1%.

In [ ]:
def test_model(model, sequence_length=15):
  """
  This is the test function that runs 100 different strings through your model,
  and checks the error rate.
  """
  total = 0
  correct = 0
  for i in range(500):
    s = ''.join([random.choice(string.ascii_lowercase) for i in range(random.randint(15, 25))])
    result = model.test_run(s)
    D = 4
    assert D > 0, 's[:-D] won\'t work for D=0'
    for c1, c2 in zip(s[:-D], result[D:]):
      correct += int(c1 == c2)
    total += len(s) - D

  return correct / total

In [ ]:
import time
from tqdm.notebook import tqdm

start_time = time.time()

# TODO: initialize and train your model here.
def train(model, batch, criterion, optimizer, device):
  input = idx_to_onehot(batch[0]).to(device)
  target = idx_to_onehot(batch[1]).to(device)

  logits = model(input)
  loss = criterion(logits, target)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

BATCH_SIZE = 50
train_dataloader = torch.utils.data.DataLoader(ds, batch_size=BATCH_SIZE)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

h_units = 5*27
model = GRUMemory(h_units).to(device)

criterion = torch.nn.BCELoss().to(device)
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001)

N_EPOCHS = 5
pbar = tqdm(range(N_EPOCHS))
model.train()
for i in pbar:
  for batch in tqdm(train_dataloader, leave=False):
    train(model, batch, criterion, optimizer, device)

## ---- end of training ---- ##

end_time = time.time()
duration = end_time - start_time

#### test #######
accuracy = test_model(model)
assert duration < 600, 'execution took f{duration:.2f} seconds, which longer than 10 mins'
assert accuracy > 0.99, f'accuracy is too low, got {accuracy}, need 0.99'
print(accuracy)
print('tests passed')


1.0
tests passed


## Variable delay model

Now, to make this more complicated, we want to have varialbe delay. So, now, the goal is to transform a sequence of pairs (character, delay) into a character sequence with given delay. Delay stays constant within one sequence.

### Dataset
As before, we first implement the dataset:

In [ ]:
import random
import string
import torch
import torch.nn as nn

N = 26 

class VariableDelayEchoDataset(torch.utils.data.IterableDataset):

  def __init__(self, max_delay=8, seq_length=20, size=1000):
    self.max_delay = max_delay
    self.seq_length = seq_length
    self.size = size
  
  def __len__(self):
    return self.size

  def __iter__(self):
    for _ in range(self.size):
      seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
      delay = random.randint(0, self.max_delay)
      result = torch.cat((torch.zeros(delay), seq[:self.seq_length - delay])).type(torch.int64)
      yield seq, delay, result

DATASET_SIZE = 200000
vds = VariableDelayEchoDataset(size=DATASET_SIZE)

In [ ]:
for a in vds:
  print(a[0])
  print(a[1])
  print(a[2])
  break

tensor([19, 26,  1, 22, 13, 20,  3, 11,  1, 18,  5,  8,  7, 13, 23,  3, 22, 26,
        14, 23])
1
tensor([ 0, 19, 26,  1, 22, 13, 20,  3, 11,  1, 18,  5,  8,  7, 13, 23,  3, 22,
        26, 14])


### Model

And the model.

In [ ]:
class VariableDelayGRUMemory(torch.nn.Module):

  def __init__(self, hidden_size, max_delay):
    super().__init__()
    #TODO
    self.hidden_size = hidden_size
    self.max_delay = max_delay
    
    self.gru = nn.GRU(
      input_size=N+1,
      hidden_size=hidden_size,
      num_layers=1,
      batch_first=True
    )
    
    self.fc = nn.Linear(
      in_features=hidden_size,
      out_features=N+1
    )

  def forward(self, x, delays):
    # inputs:
    # x - tensor of shape (batch size, seq length, N + 1)
    # delays - tensor of shape (batch size)
    # returns:
    # logits (scores for softmax) of shape (batch size, seq_length, N + 1)

    # TODO
    delays = idx_to_onehot(delays, k=self.hidden_size)  # [batch_size, self.hidden_size]   
    delays = delays.to(torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))
    h0 = delays.unsqueeze(0) # [1, batch_size, self.hidden_size]

    output, hn = self.gru(x, h0)  # output:[batch_size, seq_len, hidden_size=9*27]
    logits = self.fc(output) # [batch_size*seq_len, N+1]

    return logits # [batch_size, seq_len, N+1]

  @torch.no_grad()
  def test_run(self, s, delay):
    # This function accepts one string s containing lowercase characters a-z, 
    # and a delay - the desired output delay.
    # You need to map those characters to one-hot encodings, 
    # then get the result from your network, and then convert the output 
    # back to a string of the same length, with 0 mapped to ' ', 
    # and 1-26 mapped to a-z.

    # TODO
    x = torch.empty(0)
    for i in range(len(s)):
      tmp = ord(s[i])-96
      x = torch.cat((x,torch.tensor([tmp])))
    input = idx_to_onehot(x, N+1)
    input = input.unsqueeze(0).to(torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))     # [1,seq_len, 27]

    delays = torch.tensor([delay])    # torch.Size([1])
    output = self.forward(input, delays) # input: [1, seq_len, N+1]
    logits = torch.sigmoid(output)
    logits = logits.squeeze() # [seq_len, N+1]
    score, predicted = torch.max(logits, 1) # [seq_len]

    res = ""
    for i in range(len(predicted)):
        if predicted[i] == 0:
            res += " "
        else:
            res += chr(predicted[i]+96)
    return res

### Train

As before, you're free to do what you want, as long as training finishes within 10 minutes and accuracy is above 0.99 for delays between 0 and 8.

In [ ]:
def test_variable_delay_model(model, seq_length=20):
  """
  This is the test function that runs 100 different strings through your model,
  and checks the error rate.
  """
  total = 0
  correct = 0
  for i in range(500):
    s = ''.join([random.choice(string.ascii_lowercase) for i in range(seq_length)])
    d = random.randint(0, model.max_delay)
    result = model.test_run(s, d)
    if d > 0:
      z = zip(s[:-d], result[d:])
    else:
      z = zip(s, result)
    for c1, c2 in z:
      correct += int(c1 == c2)
    total += len(s) - d

  return correct / total

In [ ]:
import time
from tqdm.notebook import tqdm
start_time = time.time()

MAX_DELAY = 8
SEQ_LENGTH = 20

# TODO: implement model training here.
def train_variable_delay_model(model, batch, criterion, optimizer, device):
  input = idx_to_onehot(batch[0]).to(device)
  delays = batch[1]  # torch.Size([batch_size])
  target = idx_to_onehot(batch[2]).to(device)

  logits = model(input, delays)
  loss = criterion(logits, target)

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

train_dataloader2 = torch.utils.data.DataLoader(vds, batch_size=50)
device2 = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# h_units = (MAX_DELAY+1)*16 # 144
h_units = 27*9
model2 = VariableDelayGRUMemory(h_units, MAX_DELAY).to(device2)
criterion2 = torch.nn.BCEWithLogitsLoss().to(device2)
# optimizer2 = torch.optim.RMSprop(model2.parameters(), lr=0.001)
optimizer2 = torch.optim.Adam(model2.parameters(), lr=0.001, weight_decay=1e-5)
model2.train()
pbar = tqdm(range(10))
for i in pbar:
  for batch in tqdm(train_dataloader2, leave=False):
    train_variable_delay_model(model2, batch, criterion2, optimizer2, device2)
  print(i)
  print(time.time()-start_time)

## ---- end of training ---- ##

end_time = time.time()
accuracy = test_variable_delay_model(model2)
print(accuracy)
print(end_time - start_time)
assert end_time - start_time < 600, 'executing took longer than 10 mins'
assert accuracy > 0.99, f'accuracy is too low, got {accuracy}, need 0.99'
print('tests passed')

0
21.338878631591797


1
42.536176443099976


2
63.63106894493103


3
84.69064807891846


4
105.85885643959045


5
126.9726927280426


6
147.9646143913269


7
169.10550045967102


8
190.12784504890442


9
211.28820371627808

1.0
211.29046392440796
tests passed
